In [19]:
import pandas as pd

all_data=pd.read_excel('data.xlsx') #Make sure the columns donot contain trailing spaces.
all_data

data=all_data[['Service Line','Service','SOP Reference','Task/Service/Reports','Effort','Frequency_Neat','Frequency','Multiplier','Monthly Hours']]
data.drop(data[data['Service'].isna()==True].index, inplace = True)

def get_nextmonth_date_details():
    import calendar
    import datetime

    current_month=datetime.datetime.today().month+1
    current_day=datetime.datetime.today().day
    current_year=datetime.datetime.today().year
    
    _, num_days = calendar.monthrange(current_year, current_month)
    first_day = datetime.date(current_year, current_month, 1)
    last_day = datetime.date(current_year, current_month, num_days)
    
    def workdays(d, end, excluded=(6, 7)):
        working_days=[]
        day_week=[]
        working_days_week_dic={}
        day_dict={1:'Monday',2:'Tuesday',3:'Wednesday',4:'Thursday',5:'Friday'}
        while d <= end:
            if d.isoweekday() not in excluded:
                working_days.append(str(d))
                working_days_week_dic[str(d)]=day_dict[d.isoweekday()]
                day_week.append(day_dict[d.isoweekday()])
            d += datetime.timedelta(days=1)
        return(working_days,working_days_week_dic,day_week)
    
    working_days,working_days_week_dic,day_week=workdays(first_day,last_day)
    
    ind=list(range(1,len(working_days)+1))
    working_days_dic=dict(zip(ind,working_days))
    ind_day_dic=dict(zip(ind,day_week))
    return(working_days_dic,working_days,working_days_week_dic,ind_day_dic)

def get_date_details():
    import calendar
    import datetime

    current_month=datetime.datetime.today().month
    current_day=datetime.datetime.today().day
    current_year=datetime.datetime.today().year
    
    _, num_days = calendar.monthrange(current_year, current_month)
    first_day = datetime.date(current_year, current_month, 1)
    last_day = datetime.date(current_year, current_month, num_days)
    
    def workdays(d, end, excluded=(6, 7)):
        working_days=[]
        day_week=[]
        working_days_week_dic={}
        day_dict={1:'Monday',2:'Tuesday',3:'Wednesday',4:'Thursday',5:'Friday'}
        while d <= end:
            if d.isoweekday() not in excluded:
                working_days.append(str(d))
                working_days_week_dic[str(d)]=day_dict[d.isoweekday()]
                day_week.append(day_dict[d.isoweekday()])
            d += datetime.timedelta(days=1)
        return(working_days,working_days_week_dic,day_week)
    
    working_days,working_days_week_dic,day_week=workdays(first_day,last_day)
    
    ind=list(range(1,len(working_days)+1))
    working_days_dic=dict(zip(ind,working_days))
    ind_day_dic=dict(zip(ind,day_week))
    return(working_days_dic,working_days,working_days_week_dic,ind_day_dic)

working_days_dic,working_days,working_days_week_dic,ind_day_dic=get_date_details()

def get_week_day_dates(working_days_week_dic):
    keys_list,values_list=[],[]
    for i in working_days_week_dic.keys():
        keys_list.append(i)
        values_list.append(working_days_week_dic[i])
        
    monday_indlist=[]
    tuesday_indlist=[]
    wednesday_indlist=[]
    thursday_indlist=[]
    friday_indlist=[]
    for ind,i in enumerate(values_list):
        if i =='Monday':
            monday_indlist.append(ind+1)
        elif i == 'Tuesday':
            tuesday_indlist.append(ind+1)
        elif i == 'Wednesday':
            wednesday_indlist.append(ind+1)
        elif i == 'Thursday':
            thursday_indlist.append(ind+1)
        elif i == 'Friday':
            friday_indlist.append(ind+1)
    #print(monday_indlist,tuesday_indlist,wednesday_indlist,thursday_indlist,friday_indlist)
    total_list=[monday_indlist,tuesday_indlist,wednesday_indlist,thursday_indlist,friday_indlist]
    monday_list=[]
    tuesday_list=[]
    wednesday_list=[]
    thursday_list=[]
    friday_list=[]
    for ind,i in enumerate(total_list):
        #print(ind)
        for j in i:
            #print(keys_list[j-1])
            if ind ==0:
                monday_list.append(keys_list[j-1])
            elif ind == 1:
                tuesday_list.append(keys_list[j-1])
            elif ind == 2:
                wednesday_list.append(keys_list[j-1])
            elif ind == 3:
                thursday_list.append(keys_list[j-1])
            elif ind == 4:
                friday_list.append(keys_list[j-1])
    #print(monday_list,tuesday_list,wednesday_list,thursday_list,friday_list)
    week_day_dict={'Monday':monday_list,
                 'Tuesday':tuesday_list,
                 'Wednesday':wednesday_list,
                  'Thursday':thursday_list,
                  'Friday':friday_list
                 }
    return(week_day_dict)
week_day_dict=get_week_day_dates(working_days_week_dic)
count={
'Daily':len(working_days),
'Monday':len(week_day_dict['Monday']),
'Tuesday':len(week_day_dict['Tuesday']),
'Wednesday':len(week_day_dict['Wednesday']),
'Thursday':len(week_day_dict['Thursday']),
'Friday':len(week_day_dict['Friday'])}

df = pd.DataFrame(columns = data.columns) 
#Values should not contain X+10 or X+9 like that. only one single value.
next_working_days_dic,next_working_days,next_working_days_week_dic,next_ind_day_dic=get_nextmonth_date_details()

for i in data.iterrows():
    #print(i[1]['Frequency_Neat'])
    try:
        if i[1]['Frequency_Neat']=='Daily':
            for j in range(count[i[1]['Frequency_Neat']]):
                #print(j,"--",i[0])
                srs= pd.Series(data.iloc[i[0]])
                srs['Date']=working_days_dic[j+1]
                srs=srs.to_frame().T
                df = df.append(srs)
                
        elif i[1]['Frequency_Neat'] in ['Monday','Tuesday','Wednesday','Thursday','Friday']:
             for j in range(count[i[1]['Frequency_Neat']]):
                #print(j,"--",i[0])
                for p,q in zip(['Monday','Tuesday','Wednesday','Thursday','Friday'],[j for i,j in week_day_dict.items()]):
                    if i[1]['Frequency_Neat'] ==p:
                        srs= pd.Series(data.iloc[i[0]])
                        for k_ind,k in enumerate(q):
                            if j==k_ind:
                                srs['Date']=k
                                srs=srs.to_frame().T
                                df = df.append(srs)
        else:
            if ('+' in i[1]['Frequency_Neat']):
                m,_,_,_=get_date_details()
                if int(i[1]['Frequency_Neat'].split('+')[1]) > max(m.keys()):
                    diff=int(i[1]['Frequency_Neat'].split('+')[1])-max(m.keys())
                    next_working_days_dic,_,_,_=get_nextmonth_date_details()
                    srs= pd.Series(data.iloc[i[0]])
                    srs['Date']=next_working_days_dic[diff]
                    srs=srs.to_frame().T
                    df = df.append(srs)                
                
                elif i[1]['Frequency_Neat'].split('+')[1] != '0':
                    #print(i[1]['Frequency_Neat'].split('+')[1],'  ',working_days_dic[int(i[1]['Frequency_Neat'].split('+')[1])])
                    srs= pd.Series(data.iloc[i[0]])
                    srs['Date']=working_days_dic[int(i[1]['Frequency_Neat'].split('+')[1])]
                    srs=srs.to_frame().T
                    df = df.append(srs)
                elif i[1]['Frequency_Neat'].split('+')[1] == '0':
                    #print(i[1]['Frequency_Neat'].split('+')[1])
                    srs= pd.Series(data.iloc[i[0]])
                    srs['Date']='Adhoc'
                    srs=srs.to_frame().T
                    df = df.append(srs)
                
                
                
                    
            elif ('-' in i[1]['Frequency_Neat']):
                #print("-----",i[1]['Frequency_Neat'].split('-')[1],'=======',str(max(a.keys())-int(i[1]['Frequency_Neat'].split('-')[1])))
                a,_,_,_=get_nextmonth_date_details()
                
                srs= pd.Series(data.iloc[i[0]])
                srs['Date']=working_days_dic[int(max(a.keys())-int(i[1]['Frequency_Neat'].split('-')[1]))]
                srs=srs.to_frame().T
                df = df.append(srs)
        
    except:
        print("exception")
        raise
df.reset_index(drop=True,inplace=True) 



def get_task_details(df,val):
    
    df['New Tasks'] = df['SOP Reference'].fillna('No SOP').str.cat(df['Task/Service/Reports'],sep="  -->  ").str.cat(df['Frequency'],sep="  -->  ")
    
    task_details=""
    for i,j in df[df['Date'].isin([val])].filter(items=['New Tasks']).items():
        for k in j:
            #print(k)
            task_details+=k
            task_details+='\n'
    return(task_details)


def get_task(df,val):
    tasks=""
    for i,j in df[df['Date'].isin([val])].filter(items=['Task/Service/Reports']).items():
        for k in j:
            #print(k)
            tasks+=k
            tasks+='\n'
    return(tasks)
    
def send_email(body, subject, recipient,signature, auto=True):
    import win32com.client as win32   

    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = recipient
    mail.Subject = subject
    
    body=body+'<br>'*1+signature
    mail.HtmlBody = body
    if auto:
        mail.send
    else:
        mail.open 
def show_draft(body, subject, recipient,signature, auto=True):
    import win32com.client as win32   

    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = recipient
    mail.Subject = subject
    mail.HtmlBody = body
    
    body=body+'<br>'*1+signature
    mail.HtmlBody = body
    if auto:
        mail.display()
    else:
        mail.open
        
import PySimpleGUI as sg
import sys
import os
import docx
import time
import win32com.client as win32   

import time
named_tuple = time.localtime()
time_string=time.strftime("%Y-%m-%d",named_tuple)
time_string

sg.theme('DarkBlue17') 

layout = [ 
          [sg.Text('Task Reminder', size=(36, 1), justification='center', font=("Helvetica",20), relief=sg.RELIEF_RIDGE)],    
     [sg.Text('Service Line Name: '),sg.InputCombo(list(df['Service Line'].unique()),key='Service Line', size=(20, 2), enable_events=True),
      sg.Text('Service Name: '),sg.InputText('', key='Service',size=(20, 10),enable_events=True)],       
    [sg.Text('Date List:', pad=((60,10),(0,0))), (sg.InputCombo(working_days,key='date2', size=(20, 20), enable_events=True)),
                sg.Text('Date:', pad=((61,0),(0,0))),sg.InputText(time_string, key='date',size=(20, 10), pad=((10,50),(0,0)),enable_events=True)],
    [sg.Button('Show Task Details',size=(20,1)),sg.Button('Show Tasks',size=(20,1))],
            [sg.Text('Tasks-')],[ sg.Multiline('', key='tasks',size=(80, 25))],       
            
    [sg.Button('Send Mail'),sg.Button('Show Draft')]
         ]

window = sg.Window('Remind Me',resizable=True).Layout(layout)
while True:
    event, values = window.read()
    if event == 'Show Tasks':
        if len(values['Service']) <3:
            sg.popup("Alert","Please enter a service value before looking for tasks.")
        window['tasks'].update(get_task(df[df['Service']==values['Service']],values['date']))
        
    if event == 'Show Task Details':
        window['tasks'].update(get_task_details(df[df['Service']==values['Service']],values['date']))

    if event == 'Service Line':
        layout2=[ [sg.Text('Service Name: ')
                 ,sg.InputCombo(list(df[df['Service Line']==values['Service Line']]['Service'].unique()),key='Service2', size=(20, 10), enable_events=True)]]
        window2=sg.Window('Choose Service Name',resizable=True).Layout(layout2)
        event2, values2 = window2.read()
        if event2 == 'Service2':
            window['Service'].update(values2['Service2'])
            window2.close()
    
    if event == 'date2':
        window['date'].update(values['date2'])
    
    
    if event == 'Show Draft':
        if len(values['Service']) <3:
            sg.popup("Alert","Please enter a service value before looking for tasks.")
        else:
            body="""Hello All,
    
            Please make sure the tasks for """+time_string+""" are completed by EOD. \nTasks for the day are :\n\n"""+get_task(df[df['Service']==values['Service']],values['date'])
            
            subject="Reminder for Tasks "+time_string+" for "+values['Service']+"."
            
            signature="Regards,<br>Asset Management Team"
            recipient=";".join(mail_dic[values['Service']])+";VK00489900@techmahindra.com"
            show_draft(body.replace('\n','<br>'), subject, recipient,signature, auto=True)
    if event == 'Send Mail':
        if len(values['Service']) <3:
            sg.popup("Alert","Please enter a service value before looking for tasks.")
        else:
            body="""Hello All,
    
            Please make sure the tasks for """+time_string+""" are completed by EOD. \nTasks for the day are :\n\n"""+get_task(df[df['Service']==values['Service']],values['date'])
            
            subject="Reminder for Tasks "+time_string+" for "+values['Service']+"."
            
            signature="Regards,<br>Asset Management Team"
            recipient=";".join(mail_dic[values['Service']])+";VK00489900@techmahindra.com"
            send_email(body.replace('\n','<br>'), subject, recipient,signature, auto=True)
        
        
    if event == sg.WIN_CLOSED:
        break

resources_df=pd.read_excel('resources_list.xlsx')
resources_df
master_service_set=set()
for i,j in resources_df['Service'].items():
    for k in j.split(','):
        master_service_set.add(k)
mail_dic={}
for i,j in resources_df['Service'].items():
    for k in j.split(','):
        if k in list(master_service_set):
            if len(mail_dic)==0:
                mail_dic[k]=[resources_df['Email'][i]]
                #print(mail_dic)
                if len(mail_dic[k])==0:
                    mail_dic[k]=[resources_df['Email'][i]]
            else:
                #print("Dic",mail_dic)
                try:
                    if len(mail_dic[k])==0:
                        mail_dic[k]=[resources_df['Email'][i]]
                    else:
                        mail_dic[k].append(resources_df['Email'][i])
                except:
                    mail_dic[k]=[resources_df['Email'][i]]
        


C:\Users\vk00489900\Anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-19-2786b0af40f9>:206: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['New Tasks'] = df['SOP Reference'].fillna('No SOP').str.cat(df['Task/Service/Reports'],sep="  -->  ").str.cat(df['Frequency'],sep="  -->  ")


0      Asset Management
1      Asset Management
2      Asset Management
3      Asset Management
4      Asset Management
             ...       
126          Legal Docs
127    Asset Management
128    Asset Management
129    Asset Management
130    Asset Management
Name: Service Line, Length: 131, dtype: object

In [ ]:
import pandas as pd

all_data=pd.read_excel('data.xlsx') #Make sure the columns donot contain trailing spaces.
all_data

In [ ]:
data=all_data[['Service Line','Service','SOP Reference','Task/Service/Reports','Effort','Frequency_Neat','Frequency','Multiplier','Monthly Hours']]
data.drop(data[data['Service'].isna()==True].index, inplace = True)

In [ ]:
def get_date_details():
    import calendar
    import datetime

    current_month=datetime.datetime.today().month
    current_day=datetime.datetime.today().day
    current_year=datetime.datetime.today().year
    
    _, num_days = calendar.monthrange(current_year, current_month)
    first_day = datetime.date(current_year, current_month, 1)
    last_day = datetime.date(current_year, current_month, num_days)
    
    def workdays(d, end, excluded=(6, 7)):
        working_days=[]
        day_week=[]
        working_days_week_dic={}
        day_dict={1:'Monday',2:'Tuesday',3:'Wednesday',4:'Thursday',5:'Friday'}
        while d <= end:
            if d.isoweekday() not in excluded:
                working_days.append(str(d))
                working_days_week_dic[str(d)]=day_dict[d.isoweekday()]
                day_week.append(day_dict[d.isoweekday()])
            d += datetime.timedelta(days=1)
        return(working_days,working_days_week_dic,day_week)
    
    working_days,working_days_week_dic,day_week=workdays(first_day,last_day)
    
    ind=list(range(1,len(working_days)+1))
    working_days_dic=dict(zip(ind,working_days))
    ind_day_dic=dict(zip(ind,day_week))
    return(working_days_dic,working_days,working_days_week_dic,ind_day_dic)

In [ ]:
working_days_dic,working_days,working_days_week_dic,ind_day_dic=get_date_details()
print(monday_indlist,tuesday_indlist,wednesday_indlist,thursday_indlist,friday_indlist)
working_days_dic

In [ ]:
def get_week_day_dates(working_days_week_dic):
    keys_list,values_list=[],[]
    for i in working_days_week_dic.keys():
        keys_list.append(i)
        values_list.append(working_days_week_dic[i])
        
    monday_indlist=[]
    tuesday_indlist=[]
    wednesday_indlist=[]
    thursday_indlist=[]
    friday_indlist=[]
    for ind,i in enumerate(values_list):
        if i =='Monday':
            monday_indlist.append(ind+1)
        elif i == 'Tuesday':
            tuesday_indlist.append(ind+1)
        elif i == 'Wednesday':
            wednesday_indlist.append(ind+1)
        elif i == 'Thursday':
            thursday_indlist.append(ind+1)
        elif i == 'Friday':
            friday_indlist.append(ind+1)
    #print(monday_indlist,tuesday_indlist,wednesday_indlist,thursday_indlist,friday_indlist)
    total_list=[monday_indlist,tuesday_indlist,wednesday_indlist,thursday_indlist,friday_indlist]
    monday_list=[]
    tuesday_list=[]
    wednesday_list=[]
    thursday_list=[]
    friday_list=[]
    for ind,i in enumerate(total_list):
        #print(ind)
        for j in i:
            #print(keys_list[j-1])
            if ind ==0:
                monday_list.append(keys_list[j-1])
            elif ind == 1:
                tuesday_list.append(keys_list[j-1])
            elif ind == 2:
                wednesday_list.append(keys_list[j-1])
            elif ind == 3:
                thursday_list.append(keys_list[j-1])
            elif ind == 4:
                friday_list.append(keys_list[j-1])
    #print(monday_list,tuesday_list,wednesday_list,thursday_list,friday_list)
    week_day_dict={'Monday':monday_list,
                 'Tuesday':tuesday_list,
                 'Wednesday':wednesday_list,
                  'Thursday':thursday_list,
                  'Friday':friday_list
                 }
    return(week_day_dict)

In [ ]:
week_day_dict=get_week_day_dates(working_days_week_dic)
week_day_dict

In [ ]:
count={
'Daily':len(working_days),
'Monday':len(week_day_dict['Monday']),
'Tuesday':len(week_day_dict['Tuesday']),
'Wednesday':len(week_day_dict['Wednesday']),
'Thursday':len(week_day_dict['Thursday']),
'Friday':len(week_day_dict['Friday'])}

In [ ]:
week_day_dict['Monday']

In [ ]:
df = pd.DataFrame(columns = data.columns) 
df

In [ ]:
val='2020-09-21'
df[df['Date'].isin([val])].filter(items=['Service Line','Service','SOP Reference','Task/Service/Reports','Frequency_Neat','Frequency','Date'])

df[df['Date'].isin([val])].filter(items=['Service Line','Service','SOP Reference','Task/Service/Reports','Frequency_Neat','Frequency','Date'])

In [ ]:
df

In [ ]:
#Values should not contain X+10 or X+9 like that. only one single value.
next_working_days_dic,next_working_days,next_working_days_week_dic,next_ind_day_dic=get_nextmonth_date_details()

for i in data.iterrows():
    #print(i[1]['Frequency_Neat'])
    try:
        if i[1]['Frequency_Neat']=='Daily':
            for j in range(count[i[1]['Frequency_Neat']]):
                #print(j,"--",i[0])
                srs= pd.Series(data.iloc[i[0]])
                srs['Date']=working_days_dic[j+1]
                srs=srs.to_frame().T
                df = df.append(srs)
                
        elif i[1]['Frequency_Neat'] in ['Monday','Tuesday','Wednesday','Thursday','Friday']:
             for j in range(count[i[1]['Frequency_Neat']]):
                #print(j,"--",i[0])
                for p,q in zip(['Monday','Tuesday','Wednesday','Thursday','Friday'],[j for i,j in week_day_dict.items()]):
                    if i[1]['Frequency_Neat'] ==p:
                        srs= pd.Series(data.iloc[i[0]])
                        for k_ind,k in enumerate(q):
                            if j==k_ind:
                                srs['Date']=k
                                srs=srs.to_frame().T
                                df = df.append(srs)
        else:
            if ('+' in i[1]['Frequency_Neat']):
                m,_,_,_=get_date_details()
                if int(i[1]['Frequency_Neat'].split('+')[1]) > max(m.keys()):
                    diff=int(i[1]['Frequency_Neat'].split('+')[1])-max(m.keys())
                    next_working_days_dic,_,_,_=get_nextmonth_date_details()
                    srs= pd.Series(data.iloc[i[0]])
                    srs['Date']=next_working_days_dic[diff]
                    srs=srs.to_frame().T
                    df = df.append(srs)                
                
                elif i[1]['Frequency_Neat'].split('+')[1] != '0':
                    #print(i[1]['Frequency_Neat'].split('+')[1],'  ',working_days_dic[int(i[1]['Frequency_Neat'].split('+')[1])])
                    srs= pd.Series(data.iloc[i[0]])
                    srs['Date']=working_days_dic[int(i[1]['Frequency_Neat'].split('+')[1])]
                    srs=srs.to_frame().T
                    df = df.append(srs)
                elif i[1]['Frequency_Neat'].split('+')[1] == '0':
                    #print(i[1]['Frequency_Neat'].split('+')[1])
                    srs= pd.Series(data.iloc[i[0]])
                    srs['Date']='Adhoc'
                    srs=srs.to_frame().T
                    df = df.append(srs)
                
                
                
                    
            elif ('-' in i[1]['Frequency_Neat']):
                #print("-----",i[1]['Frequency_Neat'].split('-')[1],'=======',str(max(a.keys())-int(i[1]['Frequency_Neat'].split('-')[1])))
                a,_,_,_=get_nextmonth_date_details()
                
                srs= pd.Series(data.iloc[i[0]])
                srs['Date']=working_days_dic[int(max(a.keys())-int(i[1]['Frequency_Neat'].split('-')[1]))]
                srs=srs.to_frame().T
                df = df.append(srs)
        
    except:
        print("exception")
        raise
df.reset_index(drop=True,inplace=True)   

In [13]:
working_days_dic


{1: '2020-09-01',
 2: '2020-09-02',
 3: '2020-09-03',
 4: '2020-09-04',
 5: '2020-09-07',
 6: '2020-09-08',
 7: '2020-09-09',
 8: '2020-09-10',
 9: '2020-09-11',
 10: '2020-09-14',
 11: '2020-09-15',
 12: '2020-09-16',
 13: '2020-09-17',
 14: '2020-09-18',
 15: '2020-09-21',
 16: '2020-09-22',
 17: '2020-09-23',
 18: '2020-09-24',
 19: '2020-09-25',
 20: '2020-09-28',
 21: '2020-09-29',
 22: '2020-09-30'}

In [ ]:
df.columns

In [ ]:

for i in data.iterrows():
    print(i[1]['Frequency_Neat'])
    try:
        if i[1]['Frequency_Neat']=='Daily':
            for j in range(count[i[1]['Frequency_Neat']]):
                #print(j,"--",i[0])
                srs= pd.Series(data.iloc[i[0]])
                srs['Date']=working_days_dic[j+1]
                srs=srs.to_frame().T
                df = df.append(srs)
                
        if i[1]['Frequency_Neat'] in ['Monday','Tuesday','Wednesday','Thursday','Friday']:
             for j in range(count[i[1]['Frequency_Neat']]):
                print(j,"--",i[0])
                for p,q in zip(['Monday','Tuesday','Wednesday','Thursday','Friday'],[j for i,j in week_day_dict.items()]):
                    if i[1]['Frequency_Neat'] ==p:
                        srs= pd.Series(data.iloc[i[0]])
                        for k_ind,k in enumerate(q):
                            if j==k_ind:
                                srs['Date']=k
                                srs=srs.to_frame().T
                                df = df.append(srs)
        
    except:
        print("exception")
        raise
    

In [ ]:

"""df['date'][j]=working_days_dic[j+1]
                df.reset_index(drop=True)"""
df.reset_index(drop=True,inplace=True)



In [ ]:
from IPython.display import display, HTML

display(HTML(df.to_html()))

In [ ]:
import pandas as pd
import datetime as dt
pd.Timestamp(dt.date(2020, 1, 29)).quarter

In [ ]:
def get_nextmonth_date_details():
    import calendar
    import datetime

    current_month=datetime.datetime.today().month+1
    current_day=datetime.datetime.today().day
    current_year=datetime.datetime.today().year
    
    _, num_days = calendar.monthrange(current_year, current_month)
    first_day = datetime.date(current_year, current_month, 1)
    last_day = datetime.date(current_year, current_month, num_days)
    
    def workdays(d, end, excluded=(6, 7)):
        working_days=[]
        day_week=[]
        working_days_week_dic={}
        day_dict={1:'Monday',2:'Tuesday',3:'Wednesday',4:'Thursday',5:'Friday'}
        while d <= end:
            if d.isoweekday() not in excluded:
                working_days.append(str(d))
                working_days_week_dic[str(d)]=day_dict[d.isoweekday()]
                day_week.append(day_dict[d.isoweekday()])
            d += datetime.timedelta(days=1)
        return(working_days,working_days_week_dic,day_week)
    
    working_days,working_days_week_dic,day_week=workdays(first_day,last_day)
    
    ind=list(range(1,len(working_days)+1))
    working_days_dic=dict(zip(ind,working_days))
    ind_day_dic=dict(zip(ind,day_week))
    return(working_days_dic,working_days,working_days_week_dic,ind_day_dic)

In [14]:
get_nextmonth_date_details()

({1: '2020-10-01',
  2: '2020-10-02',
  3: '2020-10-05',
  4: '2020-10-06',
  5: '2020-10-07',
  6: '2020-10-08',
  7: '2020-10-09',
  8: '2020-10-12',
  9: '2020-10-13',
  10: '2020-10-14',
  11: '2020-10-15',
  12: '2020-10-16',
  13: '2020-10-19',
  14: '2020-10-20',
  15: '2020-10-21',
  16: '2020-10-22',
  17: '2020-10-23',
  18: '2020-10-26',
  19: '2020-10-27',
  20: '2020-10-28',
  21: '2020-10-29',
  22: '2020-10-30'},
 ['2020-10-01',
  '2020-10-02',
  '2020-10-05',
  '2020-10-06',
  '2020-10-07',
  '2020-10-08',
  '2020-10-09',
  '2020-10-12',
  '2020-10-13',
  '2020-10-14',
  '2020-10-15',
  '2020-10-16',
  '2020-10-19',
  '2020-10-20',
  '2020-10-21',
  '2020-10-22',
  '2020-10-23',
  '2020-10-26',
  '2020-10-27',
  '2020-10-28',
  '2020-10-29',
  '2020-10-30'],
 {'2020-10-01': 'Thursday',
  '2020-10-02': 'Friday',
  '2020-10-05': 'Monday',
  '2020-10-06': 'Tuesday',
  '2020-10-07': 'Wednesday',
  '2020-10-08': 'Thursday',
  '2020-10-09': 'Friday',
  '2020-10-12': 'Monday',

In [12]:
import time
named_tuple = time.localtime()
time_string=time.strftime("%Y-%m-%d",named_tuple)
time_string

'2020-09-23'

In [101]:
df.columns

Index(['Service Line', 'Service', 'SOP Reference', 'Task/Service/Reports',
       'Effort', 'Frequency_Neat', 'Frequency', 'Multiplier', 'Monthly Hours',
       'Date'],
      dtype='object')

In [3]:
val='2020-09-21'

def get_task_details(df,val):
    
    df['New Tasks'] = df['SOP Reference'].fillna('No SOP').str.cat(df['Task/Service/Reports'],sep="  -->  ").str.cat(df['Frequency'],sep="  -->  ")
    
    task_details=""
    for i,j in df[df['Date'].isin([val])].filter(items=['New Tasks']).items():
        for k in j:
            #print(k)
            task_details+=k
            task_details+='\n'
    return(task_details)

def get_task(df,val):
    tasks=""
    for i,j in df[df['Date'].isin([val])].filter(items=['Task/Service/Reports']).items():
        for k in j:
            #print(k)
            tasks+=k
            tasks+='\n'
    return(tasks)
    



In [135]:
for i in range(535):
    #print(i,len(df['SOP Reference'][i]))
    pass
df.iloc[22]
df['SOP Reference'].fillna(0)

0

In [78]:
def send_email(body, subject, recipient,signature, auto=True):
    import win32com.client as win32   

    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = recipient
    mail.Subject = subject
    
    body=body+'<br>'*1+signature
    mail.HtmlBody = body
    if auto:
        mail.send
    else:
        mail.open 
def show_draft(body, subject, recipient,signature, auto=True):
    import win32com.client as win32   

    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = recipient
    mail.Subject = subject
    mail.HtmlBody = body
    
    body=body+'<br>'*1+signature
    mail.HtmlBody = body
    if auto:
        mail.display()
    else:
        mail.open

In [18]:
df

Service Line               Service SOP Reference  \
0    Asset Management  Customized Reporting     OPSDLY018   
1    Asset Management  Customized Reporting     OPSDLY018   
2    Asset Management  Customized Reporting     OPSDLY018   
3    Asset Management  Customized Reporting     OPSDLY018   
4    Asset Management  Customized Reporting     OPSDLY018   
..                ...                   ...           ...   
530        Legal Docs      Client Reporting     OPSADH002   
531  Asset Management                   OPF     OPSADH004   
532  Asset Management                   OPF     OPSADH005   
533  Asset Management                   OPF     OPSADH001   
534  Asset Management             Factsheet     OPSADH002   

                                  Task/Service/Reports  Effort Frequency_Neat  \
0          Allianz GI Hong Kong Reporting - Daily Pair   0.625          Daily   
1          Allianz GI Hong Kong Reporting - Daily Pair   0.625          Daily   
2          Allianz GI Hong Kong Reporting - Daily Pair   0.625          Daily   
3          Allianz GI Hong Kong Reporting - Daily Pair   0.625          Daily   
4          Allianz GI Hong Kong Reporting - Daily Pair   0.625          Daily   
..                                                 ...     ...            ...   
530  Risk Reports - SICAV Board of Directors Reporting    1.25           X+12   
531  Miscellanous Reports - AAM Performance Portal ...       0           X+12   
532      Miscellanous Reports - Solvency II: AllianzGI       0           X+12   
533                 Risk Reports - Risk Profile Report  0.9375           X+12   
534  Risk Reports - SICAV Board of Directors Reporting    1.25           X+12   

                                             Frequency Multiplier  \
0                                                Daily         21   
1                                                Daily         21   
2                                                Daily         21   
3                                                Daily         21   
4                                                Daily         21   
..                                                 ...        ...   
530                                             Ad-Hoc          1   
531  Adhoc  - as and when request arrives in PPF-De...          1   
532  Adhoc - No offshore support service requested ...          1   
533                                             Ad-Hoc          1   
534                                             Ad-Hoc          1   

    Monthly Hours        Date  
0          13.125  2020-09-01  
1          13.125  2020-09-02  
2          13.125  2020-09-03  
3          13.125  2020-09-04  
4          13.125  2020-09-07  
..            ...         ...  
530          1.25  2020-09-16  
531             0  2020-09-16  
532             0  2020-09-16  
533        0.9375  2020-09-16  
534          1.25  2020-09-16  

[535 rows x 10 columns]

In [113]:
#list(df[df['Service Line'==values['Service Line']]]['Service'].unique())
list(df[df['Service Line']==values['Service Line']]['Service'].unique())

['Customized Reporting']

In [4]:
import PySimpleGUI as sg
import sys
import os
import docx
import time
import win32com.client as win32   

import time
named_tuple = time.localtime()
time_string=time.strftime("%Y-%m-%d",named_tuple)
time_string

sg.theme('DarkBlue17') 

layout = [ 
          [sg.Text('Task Reminder', size=(36, 1), justification='center', font=("Helvetica",20), relief=sg.RELIEF_RIDGE)],    
     [sg.Text('Service Line Name: '),sg.InputCombo(list(df['Service Line'].unique()),key='Service Line', size=(20, 2), enable_events=True),
      sg.Text('Service Name: '),sg.InputText('', key='Service',size=(20, 10),enable_events=True)],       
    [sg.Text('Date List:', pad=((60,10),(0,0))), (sg.InputCombo(working_days,key='date2', size=(20, 20), enable_events=True)),
                sg.Text('Date:', pad=((61,0),(0,0))),sg.InputText(time_string, key='date',size=(20, 10), pad=((10,50),(0,0)),enable_events=True)],
    [sg.Button('Show Task Details',size=(20,1)),sg.Button('Show Tasks',size=(20,1))],
            [sg.Text('Tasks-')],[ sg.Multiline('', key='tasks',size=(80, 25))],       
            
    [sg.Button('Send Mail'),sg.Button('Show Draft')]
         ]

window = sg.Window('Remind Me',resizable=True).Layout(layout)
while True:
    event, values = window.read()
    if event == 'Show Tasks':
        if len(values['Service']) <3:
            sg.popup("Alert","Please enter a service value before looking for tasks.")
        window['tasks'].update(get_task(df[df['Service']==values['Service']],values['date']))
        
    if event == 'Show Task Details':
        window['tasks'].update(get_task_details(df[df['Service']==values['Service']],values['date']))

    if event == 'Service Line':
        layout2=[ [sg.Text('Service Name: ')
                 ,sg.InputCombo(list(df[df['Service Line']==values['Service Line']]['Service'].unique()),key='Service2', size=(20, 10), enable_events=True)]]
        window2=sg.Window('Choose Service Name',resizable=True).Layout(layout2)
        event2, values2 = window2.read()
        if event2 == 'Service2':
            window['Service'].update(values2['Service2'])
            window2.close()
    
    if event == 'date2':
        window['date'].update(values['date2'])
    
    
    if event == 'Show Draft':
        if len(values['Service']) <3:
            sg.popup("Alert","Please enter a service value before looking for tasks.")
        else:
            body="""Hello All,
    
            Please make sure the tasks for """+time_string+""" are completed by EOD. \nTasks for the day are :\n\n"""+get_task(df[df['Service']==values['Service']],values['date'])
            
            subject="Reminder for Tasks "+time_string+" for "+values['Service']+"."
            
            signature="Regards,<br>Asset Management Team"
            recipient=";".join(mail_dic[values['Service']])+";VK00489900@techmahindra.com"
            show_draft(body.replace('\n','<br>'), subject, recipient,signature, auto=True)
            
    if event == 'Send Mail':
        if len(values['Service']) <3:
            sg.popup("Alert","Please enter a service value before looking for tasks.")
        else:
            body="""Hello All,
    
            Please make sure the tasks for """+time_string+""" are completed by EOD. \nTasks for the day are :\n\n"""+get_task(df[df['Service']==values['Service']],values['date'])
            
            subject="Reminder for Tasks "+time_string+" for "+values['Service']+"."
            
            signature="Regards,<br>Asset Management Team"
            recipient=";".join(mail_dic[values['Service']])+";VK00489900@techmahindra.com"
            send_email(body.replace('\n','<br>'), subject, recipient,signature, auto=True)
        
        
    if event == sg.WIN_CLOSED:
        break

        

<ipython-input-3-d2543bb03f9c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['New Tasks'] = df['SOP Reference'].fillna('No SOP').str.cat(df['Task/Service/Reports'],sep="  -->  ").str.cat(df['Frequency'],sep="  -->  ")


In [110]:
resources_df=pd.read_excel('resources_list.xlsx')
resources_df

Service Line                                          Service  \
0   Asset Management    OTC,OPF,Client Reporting,Customized Reporting   
1   Asset Management        OPF,Client Reporting,Customized Reporting   
2   Asset Management           APA,Factsheet,OPF,Regulatory Reporting   
3   Asset Management               Factsheet,OPF,Regulatory Reporting   
4   Asset Management  Factsheet,Customized Reports,Sales Prospectuses   
5         Legal Docs                               Sales Prospectuses   
6   Asset Management                               Customized Reports   
7   Asset Management                               Customized Reports   
8   Asset Management                                              OTC   
9   Asset Management                                        Factsheet   
10  Asset Management                                        APA,TOPAS   
11  Asset Management                                        Factsheet   
12  Asset Management                                      BAFIN,TOPAS   
13        Legal Docs                                     All services   
14        Legal Docs                                     All services   
15        Legal Docs                                     All services   
16        Legal Docs                                     All services   
17  Asset Management                                      TOPAS,BAFIN   
18  Asset Management                                     All services   
19        Legal Docs                                     All services   

   BENSL_ID                   Name                   Role  \
0   M5SUWQ6    Abhishek Raikhelkar      Software Engineer   
1   H6EN9IR        Goury Nirmaliya       Business Analyst   
2   WW4E42R              Keerthana       Business Analyst   
3   NRNX4CP        Navami HIremath      Software Engineer   
4   IP9ESA7          Robin Tripath      Software Engineer   
5   IP9ESA7          Robin Tripath      Software Engineer   
6   KF2JVGU          Ragul Aaditya      Software Engineer   
7   HTSH4VM           Priya Mishra      Software Engineer   
8   EEFJT9B    Radhika Raviprakash       Business Analyst   
9   ZMD49WF       Vinayak Kulkarni       Business Analyst   
10  J7VKWCN      Dhanunjaya Gaddam      Software Engineer   
11  EDDUZGR               Pooja RG      Software Engineer   
12  NA3X5KZ        Harish Kummathi      Software Engineer   
13  UQ2JVM3    Madhuchhanda Dehuri      Software Engineer   
14  J8VUYM6        Krishna Mohanty      Software Engineer   
15  N5MTJGC  Viswanath Reddy Peram      Software Engineer   
16  ZYT2FW7          Srujan Kantam      Software Engineer   
17  YDEWE58    Jaganathan Lingaraj              Tech Lead   
18  GQQSKF7        Sudhagar Joghee  Sr. Software Engineer   
19  GQQSKF7        Sudhagar Joghee  Sr. Software Engineer   

                            Phone  \
0                  -91 8310920497   
1                     17838645084   
2                 +91 19944890809   
3                   91 9035800105   
4                  -91 7980465358   
5                  -91 7980465358   
6                  +91 7708460685   
7                   91 8087260577   
8                   91 9844940505   
9                  +91 8208116613   
10                 +91 9160970180   
11                 +91 8792545223   
12                 +91 9441413757   
13                 +91 8722748444   
14                  91 8908487798   
15                 +91 9035635541   
16                 +91 9513388567   
17  +91 91 89409 9612 89409 96122   
18                +91 19632079427   
19                +91 19632079427   

                                                Email  
0   extern.raikhelkar_abhishek@investmentdataseric...  
1    extern.nirmalia_goury@investmentdataservices.com  
2       extern.m_keerthana@investmentdataservices.com  
3    extern.hiremath_navami@investmentdatasenices.com  
4    extern.robin_tripathi@investmentdataservices.com  
5    extern.robin_tripathi@investmentdataservices.com  
6     extern.adhitya_r

In [8]:
resources_df=pd.read_excel('resources_list.xlsx')
resources_df
master_service_set=set()
for i,j in resources_df['Service'].items():
    for k in j.split(','):
        master_service_set.add(k)
mail_dic={}
for i,j in resources_df['Service'].items():
    for k in j.split(','):
        if k in list(master_service_set):
            if len(mail_dic)==0:
                mail_dic[k]=[resources_df['Email'][i]]
                #print(mail_dic)
                if len(mail_dic[k])==0:
                    mail_dic[k]=[resources_df['Email'][i]]
            else:
                #print("Dic",mail_dic)
                try:
                    if len(mail_dic[k])==0:
                        mail_dic[k]=[resources_df['Email'][i]]
                    else:
                        mail_dic[k].append(resources_df['Email'][i])
                except:
                    mail_dic[k]=[resources_df['Email'][i]]
                    
mail_dic  

{'OTC': ['extern.raikhelkar_abhishek@investmentdataserices.com',
  'extern.raviprakash_radhika@investmentdataserices.com'],
 'OPF': ['extern.raikhelkar_abhishek@investmentdataserices.com',
  'extern.nirmalia_goury@investmentdataservices.com',
  'extern.m_keerthana@investmentdataservices.com',
  'extern.hiremath_navami@investmentdatasenices.com'],
 'Client Reporting': ['extern.raikhelkar_abhishek@investmentdataserices.com',
  'extern.nirmalia_goury@investmentdataservices.com'],
 'Customized Reporting': ['extern.raikhelkar_abhishek@investmentdataserices.com',
  'extern.nirmalia_goury@investmentdataservices.com'],
 'APA': ['extern.m_keerthana@investmentdataservices.com',
  'extemn.gaddam_chanunjaya@investmentdataservices.com'],
 'Factsheet': ['extern.m_keerthana@investmentdataservices.com',
  'extern.hiremath_navami@investmentdatasenices.com',
  'extern.robin_tripathi@investmentdataservices.com',
  'extern.kulkarni_vinayak@investmentdataservices.com',
  'extern.rg_pooja@investmentdataserv

In [10]:
resources_df=pd.read_excel('resources_list.xlsx')
resources_df
master_name_set=set()
for i,j in resources_df['Service'].items():
    for k in j.split(','):
        master_name_set.add(k)
name_dic={}
for i,j in resources_df['Service'].items():
    for k in j.split(','):
        if k in list(master_name_set):
            if len(name_dic)==0:
                name_dic[k]=[resources_df['Name'][i]]
                #print(mail_dic)
                if len(name_dic[k])==0:
                    name_dic[k]=[resources_df['Name'][i]]
            else:
                #print("Dic",mail_dic)
                try:
                    if len(name_dic[k])==0:
                        name_dic[k]=[resources_df['Name'][i]]
                    else:
                        name_dic[k].append(resources_df['Name'][i])
                except:
                    name_dic[k]=[resources_df['Name'][i]]
                    
name_dic

{'OTC': ['Abhishek Raikhelkar', 'Radhika Raviprakash'],
 'OPF': ['Abhishek Raikhelkar',
  'Goury Nirmaliya',
  'Keerthana',
  'Navami HIremath'],
 'Client Reporting': ['Abhishek Raikhelkar', 'Goury Nirmaliya'],
 'Customized Reporting': ['Abhishek Raikhelkar', 'Goury Nirmaliya'],
 'APA': ['Keerthana', 'Dhanunjaya Gaddam'],
 'Factsheet': ['Keerthana',
  'Navami HIremath',
  'Robin Tripath',
  'Vinayak Kulkarni',
  'Pooja RG'],
 'Regulatory Reporting': ['Keerthana', 'Navami HIremath'],
 'Customized Reports': ['Robin Tripath', 'Ragul Aaditya', 'Priya Mishra'],
 'Sales Prospectuses': ['Robin Tripath', 'Robin Tripath'],
 'TOPAS': ['Dhanunjaya Gaddam', 'Harish Kummathi', 'Jaganathan Lingaraj'],
 'BAFIN': ['Harish Kummathi', 'Jaganathan Lingaraj'],
 'All services': ['Madhuchhanda Dehuri',
  'Krishna Mohanty',
  'Viswanath Reddy Peram',
  'Srujan Kantam',
  'Sudhagar Joghee',
  'Sudhagar Joghee']}

In [13]:
",".join(name_dic['OTC'])

'Abhishek Raikhelkar,Radhika Raviprakash'

In [116]:
mail_dic['Factsheet']

['extern.m_keerthana@investmentdataservices.com',
 'extern.hiremath_navami@investmentdatasenices.com',
 'extern.robin_tripathi@investmentdataservices.com',
 'extern.kulkarni_vinayak@investmentdataservices.com',
 'extern.rg_pooja@investmentdataservices.com']